# GRO620 - Problématique

Voici le fichier de départ de la problématique. Si tout a été installé correctement, vous devriez voir apparaître la première image (DSCF8010.jpeg).

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

print("GRO620 - Problématique")
print("OpenCV version", cv2.__version__)

%matplotlib inline

In [ ]:
## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


In [ ]:
images_fn = os.listdir("photos_prob/")
print("%i photo(s) à traiter"%(len(images_fn)))
if (len(images_fn) == 0):
    print("ERREUR! Vérifiez que vous avez bien un dossier photos_prob au même endroit que ce calepin.")
    
images = []

for f in images_fn:
    img = cv2.imread(os.path.join("photos_prob/", f))
    images.append(img)

# Identification du problème 


Pour la problématique, l'employeur demande à faire une preuve de concept pour trier de la quincaillerie. Le mandat est de faire la séparation et l'analyse des différentes vis qui se situe sur un convoyeur.

Pour cette problématique, il y a de nombreux problèmes a résoudre. Pour commencer, il y a différente grandeur de vis, il va donc être important de différencier la taille de chacune des vis présentes. Il faut aussi identifier leur position et leur angle sur le convoyeur.

Pour le convoyeur, sa surface n'est pas parfaite, il y a donc des imperfections dans la fond de l'image prise par la caméra. De plus, le convoyeur possède des encoches directement sur sa surface, il faudra donc que la solution prenne en considération ces deux détails pour bien isoler les vis dans l'image.

Pour la caméra, elle est centré avec le convoyeur et prend des images en couleur vu de haut. L'analyse est effectué en jouant sur les paramètres des pixels de l'image. Pour trouver l'emplacement des vis sur la convoyeur, il faudra s'assurer de convertir les positions obtenues après analyse pour les mettre dans le repère du convoyeur.

In [ ]:
f = 0.023 #Distance focale (m)
res = np.array([640,427]) #Résolution de la caméra (pixel)
capt = np.array([0.0234, 0.0156]) #Dimension du capteur (m)

fx = f*res[0]/capt[0]
fy = f*res[1]/capt[1]

###Matrice de transformation convoyeur à caméra###
Tc = np.array([[1,0,0,0.5],
               [0,-1,0,0.2],
               [0,0,-1,0.282],
               [0,0,0,1]])
Tc_inv = np.linalg.inv(Tc)

###Matrice de calibration pour la caméra###
K = np.array([[fx, 0., res[0]/2],
              [0., fy, res[1]/2],
              [0., 0., 1.]])

K_p = np.identity(4)
K_p[:3, :3] = K

###Matrice de la caméra###
P = np.dot(K_p,Tc_inv)

Affichons maintenant la première image à traiter:

In [ ]:
plt.imshow(images[0])

# Procédure de résolution

Pour isoler les vis sur le convoyeur, il faut enlever les détails du convoyeur, c'est-à-dire le fond de l'image qui est rugeux. Pour ce faire, il faudra probablement utiliser un filtre qui enlevera les hautes-fréquences en prennant la valeur des couleurs de ses voisins. Le résultat devrait amener l'image avec moins de détails et un peu embrouillé.  

Le fond peut être retiré en fonction de l'opacité, car on cherche seulement les vis. Le résultat souhaité est d'avoir seulement les vis sur l'image.

Par la suite, il faut s'assurer d'avoir la bonne forme de vis, semblable à l'image de départ. Certaines transformées des pixels isolées peuvent être utilisées pour modifier l'apparence des vis.

Une fois les vis bien isolées et dimensionnées, une méthode pour trouver les contours peut être utilisée. Ayant les contours, les vis peuvent être encadrées avec un rectangle pour trouver la longueur, l'angle et le centre de la vis.

Il faut faire attention aux abérations, c'est-à-dire les fentes du convoyeur et les vis cachées sur les côtés. Il faut mettre une vérification pour trouver si le contour trouvé est vraiment une vis.

Finalement, il faut convertir les données de la vis du repère de la caméra à celui du convoyeur. Il faut ajouter une composante en z, car l'analyse de l'image donne des coordonnés en 2D. Les données sont enregistrées en csv dans un tableau contenant l'identifiant, le type, la position en X, Y, Z et l'angle en radians.

In [ ]:
def typeVis(size):
    
    itemLength = np.abs(size).max()
    
    if itemLength < 100: itemType = "courte"
    else: itemType = "longue"
    
    return str(itemType) 

In [ ]:
def repositionnementAngle(width,height,angle):
    
    if (width < height):
        angle -= 90

    return int(angle)*-np.pi/180
    

# Mise en œuvre du pipeline

La première étape est de convertir l'image en monochrome pour faciliter le travail de cette dernière.

La deuxième étape est de réduire les détails, causés par les hautes-fréquences, avec un filtre linéaire de type gaussien. Cette étape permet d'embrouiller l'image et ainsi retirer les petits détails qui influenceraient les prochaines étapes.

La troisième étape est d'utiliser un seuillage (threshold) pour enlever le fond de l'image et ainsi isoler les vis. Ceci va permet de trouver les contours plus facilement en retirant toutes les sections qui ne sont pas à analyser. Selon les valeurs choisies, les vis peuvent être affecté par le seuillage.

La quatrième étape est d'ajuster les vis isolées par le seuillage, car les limites du seuillage ne sélectionne pas la vis en entier à cause des différentes intensités de lumière. Les vis se retrouvent alors amincis et l'opération morphologique `dilate` doit être utilisé pour rétablir leur taille.

La cinquième étape est d'utiliser un filtre Canny qui détecte les points situés sur le contour des objets et fait une approximation du contour de l'objet.

La sixième étape est d'utiliser `findContours` pour sélectionner les points trouvés par la dernière étape et les enregistrer.

La septième étape est de dessiner les contours sur l'image originelle. L'identification du point central de la vis est aussi ajouté. En plus, l'utilisation de la fonction `minAreaRect` permet d'encadrer les vis et ainsi trouver leurs longueurs et leurs orientations.

La huitième étape est de trouver et transformer les données de chaque vis. Pour le type de vis, une vérification de la plus grande dimension de l'encadré permet de vérifier sa longueur. Pour trouver l'angle, il faut faire une conversion selon ce que la fonction `minAreaRect` retourne, car elle donne l'angle la plus proche de zéro ne prenant pas en compte l'orientation du rectangle. Pour la position centrale, il faut faire une conversion du repère de la caméra, en pixel et en 2D, à celui du convoyeur, en mètre et en 3D. Pour ce faire, il faut utiliser la matrice de la caméra qui permet de transformer les points du repère du convoyeur à celui de la caméra. Cependant, il faut s'assurer de faire une projection sur le plan XY pour ajouter cette composante au point en 2D. Il faut aussi s'assurer que le quatrième terme a une valeur de un en divisant la position par ce dernier.

La dernière étape est de sauvegarder les données sous la forme d'un tableau dans un fichier csv.

In [ ]:
######Décommentez les lignes plt pour voir les images de chaque section#######
def isolationDesContours(image):
    img = image.copy()

    ###Mettre en monochrome###
    img_mono = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ###Gaussian pour réduire les détails###
    img_gaus = cv2.GaussianBlur(img_mono,(11,11),0)
    #plt.figure()
    #plt.imshow(img_gaus, cmap='gray')

    ###Threshold pour enlever le background###
    ret,img_thresh = cv2.threshold(img_gaus, 190, 200, cv2.THRESH_BINARY)
    #plt.figure()
    #plt.imshow(img_thresh)

    ###Dilate pour bien représenter les contours###
    kernel = np.ones((3, 3), np.uint8)
    img_dilate = cv2.dilate(img_thresh, kernel, iterations = 1)
    #plt.figure()
    #plt.imshow(img_dilate)

    ###Canny pour isoler les contours###
    img_can = cv2.Canny(img_dilate,75,150)
     #plt.figure()
     #plt.imshow(img_can)

    ###Trouver les contours###
    contours, hierarchy = cv2.findContours(img_can,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    ###Dessiner les rectangles et les centres###
    for i in range(0,len(contours)):
        rec = cv2.minAreaRect(contours[i])
        x,y = rec[0]
        w,h = rec[1]
        theta = repositionnementAngle(w,h,rec[2])
        type_vis = typeVis(rec[1])
        
        if w > (4*h) or h > (4*w):
            cv2.drawContours(img, contours, i, (255, 10, 0), 1)
            cv2.circle(img, (int(x), int(y)), 3, (0,0,255),-1) 
            box = cv2.boxPoints(rec)
            box = np.int0(box)
            cv2.drawContours(img, [box], 0, (0,255,0), 2)
            
            vis.append((type_vis,x,y,theta))
       
    plt.figure()
    plt.imshow(img)

In [ ]:
def transformationRepere(vis_cal):
    P_t = np.linalg.inv(P)
    
    #projection en m selon l'axe z
    d = 1/0.282 

    x_s = np.array([vis_cal[1],vis_cal[2],1,d])
    x_s_conv = x_s/x_s[3]
    p_conv = np.dot(P_t, x_s_conv)
    vis_info.append((vis_cal[0],p_conv[0],p_conv[1],p_conv[2],vis_cal[3]))

In [ ]:
def exportCSV(vis_info):
    item_info = pd.DataFrame(vis_info, index=range(len(vis_info)), 
                             columns=['Type', 'X (m)', 'Y (m)', 'Z (m)', 'Theta (rad)'])
    item_info.index.name='id'
    item_info.to_csv(f'output/{filename}.csv', index=True)

In [ ]:
###Programme principal###
index = 1

for img in images: 
    filename = 'image_' + str(index)
    index += 1
    vis = list()
    vis_info = list()
    
    isolationDesContours(img)
    
    for vis_cal in vis :
        transformationRepere(vis_cal)
    
    exportCSV(vis_info)
    

# Analyse du résultat 

Les faiblesses de notre programme sont concentrées sur le rectangle autour des vis détectées. Ce dernier n'est pas dans la direction optimale qui suit la figure de la vis. La vis devrait être bien alignée dans l'encadrement. Dans notre cas, la pointe se retrouve souvent dans un coin du rectangle. Pour optimiser le résultat, il faudrait que la pointe de la vis se retrouve au milieu du côté de la boite.

Une autre faiblesse provient des encadrés qui ne détectent pas nécessairement la pointe. La pointe est cachée par les filtres qui sont ajoutés pour  enlever l'arrière plan. Cela signifie que les filtres sont très précis et puissants, mais ils cachent des pointes de vis qui sont trop petites et les confondent avec l'arrière-plan. 

Malgré ces quelques faiblesses, notre code répond à toutes les exigences pour résoudre la problématique. La détection, l'isolation et le classement des vis ce fait sans soucie et les informations sont traduites dans des documents CSV. La détection ne contient pas d'abération visible et ce dans les neuf images qui ont été fourni pour l'analyse. Les cas particuliers sont gérés. Par exemple, deux vis très rapprochées ou une vis qui serait majoritairement à l'extérieur de l'image seraient éliminé par le pipeline de traitement. 

On peut en conclure que ce code pourrait être utilisé comme preuve de concept pour le triage de la quincaillerie par vision.
